In [ ]:
# Implementation of code example from
# https://forum.orekit.org/t/kalman-model-estimator-usage-suggestions/1032

In [ ]:
            ODEIntegratorBuilder integrator = new DormandPrince853IntegratorBuilder(0.001, 1000, 1);
        PositionAngle angle = PositionAngle.MEAN;
        double scale = 1; // scaling factor used for orbital parameters normalization (typically set to the expected standard deviation of the position)
        NumericalPropagatorBuilder propagatorBuilder = new NumericalPropagatorBuilder(initialGuess, integrator, angle, scale);

        // Define the univariate functions for the standard deviations      
        UnivariateFunction[] lofCartesianOrbitalParametersEvolution = new UnivariateFunction[6];

        // Evolution for position error
        lofCartesianOrbitalParametersEvolution[0] = new PolynomialFunction(new double[] { 100, 0, 1e-4 });
        lofCartesianOrbitalParametersEvolution[1] = new PolynomialFunction(new double[] { 100, 1e-1, 0 });
        lofCartesianOrbitalParametersEvolution[2] = new PolynomialFunction(new double[] { 100, 0, 0 });

        // Evolution for velocity error
        lofCartesianOrbitalParametersEvolution[3] = new PolynomialFunction(new double[] { 1, 0, 1e-6 });
        lofCartesianOrbitalParametersEvolution[4] = new PolynomialFunction(new double[] { 1, 1e-3, 0 });
        lofCartesianOrbitalParametersEvolution[5] = new PolynomialFunction(new double[] { 1, 0, 0 });

        UnivariateFunction[] propagationParametersEvolution = new UnivariateFunction[] 
        { 
            new PolynomialFunction(new double[] { 10, 1, 1e-4 }), 
            new PolynomialFunction(new double[] { 1000, 0, 0 }),
        };

        double threshold = 1e-10; // threshold to identify matrix singularity
        RealMatrix cov = blse.getPhysicalCovariances(threshold);
        UnivariateProcessNoise covarianceMatricesProvider = new UnivariateProcessNoise(cov, LOFType.TNW, angle, lofCartesianOrbitalParametersEvolution, propagationParametersEvolution);
        
        // Get estimatedMeasurementParameters
        ParameterDriversList estimatedMeasurementParameters = new ParameterDriversList();

        foreach (ObservedMeasurement ob in obs)
            foreach (ParameterDriver driver in ob.getParametersDrivers().toArray())
                estimatedMeasurementParameters.add(driver);

        //foreach (DelegatingDriver delegating in propagatorBuilder.getOrbitalParametersDrivers().getDrivers().toArray())
        //    foreach (ParameterDriver driver in delegating.getRawDrivers().toArray())
        //        estimatedMeasurementParameters.add(driver);
        //foreach (DelegatingDriver delegating in propagatorBuilder.getPropagationParametersDrivers().getDrivers().toArray())
        //    foreach (ParameterDriver driver in delegating.getRawDrivers().toArray())
        //        estimatedMeasurementParameters.add(driver);

        KalmanEstimator kalman = new KalmanEstimatorBuilder()
            .addPropagationConfiguration(propagatorBuilder, covarianceMatricesProvider)
            .estimatedMeasurementsParameters(estimatedMeasurementParameters)
            .build();